In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../build/')

In [3]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [4]:
import versor as vsr

In [5]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, interact
from traitlets import link, dlink

# Drawing functions

In [6]:
def vector_mesh(vec, position=None, color='black', linewidth=2, arrow=False):
    length = vec.norm()
    line = Line(geometry=PlainGeometry(vertices=[[0,0,0], [0,length,0]],
                                   colors = [color, color]), 
            material=LineBasicMaterial(linewidth=linewidth,
                                       vertexColors='VertexColors'), 
            type='LinePieces')
    if arrow:
        cone = Mesh(geometry=CylinderGeometry(radiusTop = 0,
                                              radiusBottom = 0.1,
                                              height = 0.2,
                                              radiusSegments = 20,
                                              heightSegments = 1,
                                              openEnded = False),
                    material=LambertMaterial(color=color))
        cone.position = [0,length-0.1,0]
        line.children = [cone]
    line.quaternion = np.array(vec.unit().ratio(vsr.Vec(0,1,0)).quat()).tolist()
    if position is not None:
        line.position = position
    return line

def bivector_mesh(biv, color='gray'):
    geometry = CircleGeometry(radius=biv.norm(),
                              segments=64)
    material = LambertMaterial(color=color)
    mesh = Mesh(geometry=geometry,
                material=material)
    mesh.quaternion = np.array(biv.unit().duale().ratio(vsr.Vec(0,0,1)).quat()).tolist()
    return mesh

def trivector_mesh(tri, color='gray'):
    geometry = BoxGeometry()
    material = LambertMaterial(color=color)
    mesh = Mesh(geometry=geometry,
                material=material)
    mesh.material.wireframe = True
    mesh.position = [0.5,0.5,0.5]

    return mesh

def sphere_mesh(sph, color='gray'):
    mesh = Mesh(geometry=SphereGeometry(radius=sph1.radius(),
                                            widthSegments=64,
                                            heightSegments=64), 
                  material=LambertMaterial(color=color),
                  position=np.array(sph.pnt())[:3].tolist())
    return mesh

def point_mesh(pnt, color='gray'):
    mesh = Mesh(geometry=SphereGeometry(radius=0.1,
                                        widthSegments=32,
                                        heightSegments=32), 
              material=LambertMaterial(color=color))
    mesh.position = np.array(pnt)[:3].tolist()
    return mesh

def frame_mesh(position=None, quaternion=None, size=1, linewidth=2):
    frame = Line(geometry=PlainGeometry(vertices=[[0, 0, 0],[size, 0, 0],
                                                  [0, 0, 0],[0, size, 0],
                                                  [0, 0, 0],[0, 0, size]],
                                        colors = ['red','red', 
                                                  'green','green', 
                                                  'blue','blue']), 
             material=LineBasicMaterial(linewidth=linewidth, vertexColors='VertexColors'), 
             type='LinePieces')
    if position is not None:
        frame.position = position
    if quaternion is not None:
        frame.quaternion = quaternion
    return frame

# Figures

## Duality

In [7]:
B = vsr.Biv(1,0,0)
a = B.duale()

scene = Scene(children=[
        frame_mesh(),
        vector_mesh(a,color='blue', arrow=True),
        bivector_mesh(B,color='hotpink'),
        bivector_mesh(B,color='#eeeeee'),
#         DirectionalLight(color='#ccaabb', position=[1,1,1]),
        AmbientLight(color='#cccccc')])
camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'

display(renderer)

## Projection and Rejection

In [8]:
a = vsr.Vec(1,1,1)
B = vsr.Biv(1,0,0)
c = a.project_onto(B)
d = a.reject_from(B)

scene = Scene(children=[
        frame_mesh(),
        vector_mesh(a,color='gray', arrow=True),
        vector_mesh(c,color='gray'),
        vector_mesh(d,position=np.array(c).tolist(),color='gray'),
        bivector_mesh(B,color='hotpink'),
#         DirectionalLight(color='#ccaabb', position=[1,1,1]),
        AmbientLight(color='#cccccc')])
camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

## Rotation

In [ ]:
a = vsr.Vec(1,0,1)
B = vsr.Biv(1,0,0)
n = B.duale()
a_reflected = a.reflect(n)

scene = Scene(children=[
        frame_mesh(),
        vector_mesh(a,color='black', arrow=True),
        vector_mesh(a_reflected,color='black', arrow=True),
        vector_mesh(n,color='black', arrow=True),
        bivector_mesh(B,color='gray'),
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[1, -3, 0])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

## Rotation

In [35]:

n1 = vsr.Vec(0,1,0).unit()
# n2 = vsr.Vec(-1,1,0).unit()
n2 = n1.spin(vsr.Biv(-np.deg2rad(30) / 2,0,0).exp())

B = n1 ^ n2

R = n1 * n2

a = vsr.Vec(1,0,1).unit()
a_rot = a.reflect(n2).reflect(n1)
print(a_rot)
print(a.spin(R))
B_mesh = bivector_mesh(B,color='gray')
B_mesh.material.opacity = 0.9
B_mesh.material.transparent = True

scene = Scene(children=[
#         frame_mesh(),
        vector_mesh(n1,color='yellow'),
        vector_mesh(n2,color='cyan'),
        vector_mesh(a, color='hotpink'),
        vector_mesh(a.reflect(n2)),
        vector_mesh(a_rot, color='black'),
#         bivector_mesh(n1.unduale(),color='yellow'),
#         bivector_mesh(n2.unduale(),color='cyan'),
        B_mesh,
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

Vec: [ 0.35 0.61 0.71 ]
Vec: [ 0.35 0.61 0.71 ]


In [27]:
vsr.Vec(1,0,0).spin(R)

Vec: [ 0.5 0.87 0 ]

In [28]:
np.rad2deg(np.arccos(R[0]))

29.999999999999993

In [ ]:
@interact(x=10,y=10,z=10)
def position_camera(x,y,z):
    camera.position = [x,y,z]

In [ ]:
class Sphere(object):
    def __init__(self):
        pass
    def update(vsr_sph):
        r, p, q = vsr_sph.rpo()
        
def make_sphere():
    return Mesh(geometry = SphereGeometry(),
                material= BasicMaterial(color='red'))

def make_circle():
    return Mesh(geometry = RingGeometry(thetaSegments=64), 
              material= BasicMaterial(color='black'))

class Game(object):
    class Circle(vsr.Cir, Mesh):
        def __init__(self):
            self._color = 'black'
            super(Mesh, self).__init__(geometry = RingGeometry(thetaSegments=64), 
                          material= BasicMaterial(color=self._color))
            
        @property
        def mesh(self):
            return self._mesh
        
        def update_mesh(self):
            (r, p, q) = self.rpo()
            self.geometry.innerRadius = r
            self.geometry.outerRadius = r + 0.1
            self.position = p
            self.quaternion = q
        

## Rotation

In [24]:

n1 = vsr.Vec(0,1,0).unit()
# n2 = vsr.Vec(-1,1,0).unit()
n2 = n1.spin(vsr.Biv(-np.deg2rad(30),0,0).exp())

B = n1 ^ n2

R = n1 * n2

a = vsr.Vec(1,0,1)
a_rot = a.reflect(n2).reflect(n1)
print(a_rot)
print(a.spin(R))


scene = Scene(children=[
        frame_mesh(),
        vector_mesh(n1,color='yellow'),
        vector_mesh(n2,color='cyan'),
        vector_mesh(a, color='hotpink'),
        vector_mesh(a.reflect(n2)),
        vector_mesh(a_rot, color='black'),
#         bivector_mesh(n1.unduale(),color='yellow'),
#         bivector_mesh(n2.unduale(),color='cyan'),
        bivector_mesh(B,color='gray'),
        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

Vec: [ -0.5 0.87 1 ]
Vec: [ -0.5 0.87 1 ]


In [22]:
vsr.Vec(1,0,0).spin(R)

Vec: [ -0.5 0.87 0 ]

In [23]:
2 * np.rad2deg(np.arccos(R[0]))

119.99999999999997

In [ ]:
sph1 = vsr.Sph(vsr.Vec(0.7,0,0).null(), 1.5)
sph2 = vsr.Sph(vsr.Vec(-0.5,0,0).null(), 1.)
cir = sph1.meet(sph2)
r = cir.rot()

sphere1 = Mesh(geometry = SphereGeometry(radius=sph1.radius(),
                                        widthSegments=64,
                                        heightSegments=64), 
              material= BasicMaterial(color='red'),
              position=np.array(sph1.pnt())[:3].tolist())

sphere2 = Mesh(geometry = SphereGeometry(radius=sph2.radius(),
                                        widthSegments=64,
                                        heightSegments=64), 
              material= BasicMaterial(color='blue', wireframe=False),
              position=np.array(sph2.pnt())[:3].tolist())

circle = Mesh(geometry = RingGeometry(innerRadius=cir.radius(),
                                                outerRadius=cir.radius()+0.05,
                                                thetaSegments=64), 
              material= BasicMaterial(color='black'),
              position=np.array(cir.pnt())[:3].tolist(),
              quaternion = cir.rot().quat().tolist())



scene = Scene(children=[sphere1, sphere2, circle])
c = PerspectiveCamera(position=[10, 5, 10])
renderer = Renderer(camera=c, scene = scene, controls=[TrackballControls(controlling=c)],
                    background = 'white')
display(renderer)
@interact(circlecolor=('hotpink', 'gray', 'black'),
         sphere1color=('hotpink', 'gray', 'black'),
         sphere2color=('hotpink', 'gray', 'black'))
def set_radius(circlecolor, sphere1color, sphere2color):
    sphere1.material.color = sphere1color
    sphere2.material.color = sphere2color
    circle.material.color = circlecolor

In [ ]:
%%javascript
console.log('hallo')
Jupyter.notebook.get_selected_cell()

In [ ]:
cir = vsr.Sph(vsr.Vec(0,0,0).null(), 3).meet(vsr.Sph(vsr.Vec(1,0,0).null(), 3))
print(cir.rpo())
radius, position, quat = cir.rpo()